In [ ]:
import ee
import geemap
import numpy as np
import pandas as pd

In [12]:
def get_s2(dt, lat, lng):
    d1 = (dt - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    d2 = (dt + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

    distance_km = 1
    km_per_degree_lat = 111

    lat_offset = distance_km / km_per_degree_lat / 2
    lng_offset = distance_km / \
        (km_per_degree_lat * np.cos(np.radians(lat))) / 2

    bbox = ee.Geometry.Rectangle(
        [lng - lng_offset, lat - lat_offset, lng + lng_offset, lat + lat_offset])

    s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterBounds(bbox) \
        .filterDate(d1, d2) \
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .mosaic() \
        .clip(bbox)

    return s2

In [13]:
o2_dis = pd.read_csv('data/o2_dis.csv', parse_dates=['Datetime'])
o2_dis.head()

,GEMS.Station.Number,Depth,Datetime,%,mg/l,Water Type,Elevation,Latitude,Longitude
0,AUT00252,0.3,2012-10-09 12:00:00,115.0,10.1,Lake station,437.0,46.63092,14.12236
1,AUT00252,0.3,2012-12-19 12:00:00,79.0,9.4,Lake station,437.0,46.63092,14.12236
2,AUT00252,0.3,2013-05-14 12:00:00,118.0,10.6,Lake station,437.0,46.63092,14.12236
3,AUT00252,0.3,2013-06-26 12:00:00,105.0,8.8,Lake station,437.0,46.63092,14.12236
4,AUT00252,0.3,2013-09-25 12:00:00,108.0,9.4,Lake station,437.0,46.63092,14.12236


In [15]:
ee.Initialize()

for i, row in o2_dis.iterrows():
    s2 = get_s2(*row[['Datetime', 'Latitude', 'Longitude']])
    if s2.getInfo()['bands'] == []:
        print(f"No image found for {row['Datetime']}")
        continue
    else:
        print(f"Image found for {row['Datetime']}")
        break
        filename = row['GEMS.Station.Number'] + '-' + \
            row['Datetime'].strftime('%Y%m%d-%H%M%S') + '.tif'
        geemap.ee_export_image(
            s2, filename=filename, scale=10, region=s2.geometry().bounds().getInfo()['coordinates'][0], file_per_band=False)

No image found for 2012-10-09 12:00:00
No image found for 2012-12-19 12:00:00
No image found for 2013-05-14 12:00:00
No image found for 2013-06-26 12:00:00
No image found for 2013-09-25 12:00:00
No image found for 2013-12-03 12:00:00
No image found for 2014-03-11 12:00:00
No image found for 2014-06-23 12:00:00
No image found for 2014-09-16 12:00:00
No image found for 2014-12-09 12:00:00
No image found for 2016-03-16 12:00:00
No image found for 2016-06-02 12:00:00
No image found for 2016-09-07 12:00:00
Image found for 2016-12-05 12:00:00


In [16]:
s2.bandTypes().getInfo()

{'B1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B10': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B11': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B12': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B2': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B3': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B4': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B5': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B6': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B7': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B8': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B8A': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'B9': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535},
 'MSK_CLASSI_CIRRUS': {'type': 'Pi

In [125]:
Map = geemap.Map(center=[row['Latitude'], row['Longitude']], zoom=12)

vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(s2, vis_params, 'Sentinel-2 Image')

display(Map)

Map(center=[46.63092, 14.12236], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [134]:
s2.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [13, 46],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [13, 46],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [13, 46],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2, 2],
   'origin': [13, 46],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min

In [130]:
filename = row['GEMS.Station.Number'] + '-' + row['Datetime'].strftime('%Y%m%d-%H%M%S') + '.tif'

In [131]:
filename

'AUT00252-20161205-120000.tif'